# Lab CMBLenses

## Model

In [1]:
] activate "/home/jaimerz/julia_repos/CMBLensingInferenceTestProblem"

  Activating project at `~/julia_repos/CMBLensingInferenceTestProblem`


In [2]:
using CMBLensing, CMBLensingInferenceTestProblem, CUDA, LaTeXStrings, MCMCDiagnosticTools, Plots, ProgressMeter, Random, Zygote
ENV["LINES"] = 10;

In [3]:
using Revise
using MCHMC

In [4]:
prob = load_cmb_lensing_problem(storage = Array,
                                T = Float32,
                                Nside = 4);

MAP_joint: 100%|████████████████████████████████████████| Time: 0:00:47
  step:        20
  logpdf:      377.00
  α:           0.008522608
  CG:          2 iterations
  Linesearch:  9 bisections


In [5]:
target = CMBLensingTarget(prob)

LoadError: UndefVarError: `target` not defined

## Sample

In [6]:
eps=0.00000005
L=sqrt(770)
spl=Sampler(;eps=eps, L=L)

Sampler(Settings(MersenneTwister(0), 5.0e-8, 27.748873851023216, 0.0, 0.1931833275037836, 0.0005, 2000, 1000, 10, "LF"), MCHMC.Leapfrog)

In [7]:
init = MCHMC.Get_initial_conditions(spl, target, )

LoadError: UndefVarError: `target` not defined

In [8]:
init, sample = MCHMC.Step(spl, target, init)
sample

LoadError: UndefVarError: `target` not defined

In [9]:
samples = Sample(spl, target; num_steps=10, initial_x=prob.Ωstart)

LoadError: UndefVarError: `target` not defined

## Plotting

In [8]:
Plots.default(fmt=:png, dpi=120, size=(500,300), legendfontsize=10)

In [16]:
plot([exp(Ω.θ.r) for (Ω,) in chain], label=L"r", xlabel="step")
plot!([exp(Ω.θ.Aϕ) for (Ω,) in chain], label=L"A_\phi")

LoadError: UndefVarError: `chain` not defined

In [17]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [18]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [19]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [20]:

ps = map([(:ϕ°,:I,L"L",L"\phi^\circ"), (:f°,:E,"L\ell",L"E^\circ"), (:f°,:B,L"\ell",L"B^\circ")]) do (k1, k2, xlabel, title)
    plot(get_Cℓ(prob.Ωtrue[k1][k2]); label="true", xlabel, title)
    plot!(get_Cℓ(prob.Ωstart[k1][k2]); label="start", xlabel, title)
    plot!(get_Cℓ(chain[end][1][k1][k2]); label="last sample", xlabel, title)
end
plot(ps..., layout=(1,3), xscale=:log10, yscale=:log10, size=(1000,300), legend=:bottomleft)

LoadError: UndefVarError: `prob` not defined